In [1]:
# add local directory to import path
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
#### --- Standard imports ------
import pandas as pd
import seaborn as sns
import numpy as np

# local import
import nfl_bdb24
import nflutil

In [3]:
track_df = pd.DataFrame()
play_df = pd.read_csv('csv/plays.csv')
game_df = pd.read_csv('csv/games.csv')
player_df= pd.read_csv('csv/players.csv')
tackle_df = pd.read_csv('csv/tackles.csv')

# metric file
metric_df = pd.read_csv('interm_files/feat_tackle_metrics.csv')

# tracking data for all weeks
weeks = range(1,10)
for week in weeks:
    track_df = pd.concat([track_df, pd.read_csv(f'csv/tracking_week_{week}.csv').pipe(nflutil.transform_tracking_data)], ignore_index=True)

# Create overall metric

In [4]:
weight_gap = 1
weight_vision = 1
weight_drive = 1

metric_df['overall_metric'] = (
                                ((weight_gap * metric_df.tackle_gap_metric) + (weight_vision * metric_df.vision_metric) + (weight_drive * metric_df.drive_metric)) 
                               / (weight_gap + weight_vision + weight_drive)
                               )

metric_df

,gameId,playId,contactFrameId,tackleFrameId,frames,d_actual,d_ideal,d_eff,gap_tackle,w_carrier,...,nflId_t,displayName_t,position_t,ballCarrierId,displayName,position,drive_metric,tackle_gap_metric,vision_metric,overall_metric
0,2022090800,56,10,18,8,5.01,5.006246,0.999251,0.670298,191,...,43294,Jalen Ramsey,CB,42489,Stefon Diggs,WR,0.610615,1.000000,0.951500,0.854038
1,2022090800,80,20,26,6,4.87,3.768183,0.773754,0.367696,237,...,53532,Ernest Jones,ILB,46076,Josh Allen,QB,0.424705,1.000000,0.773754,0.732820
2,2022090800,101,38,45,7,8.01,5.029374,0.627887,2.476812,203,...,42816,Troy Hill,CB,47857,Devin Singletary,RB,0.408920,0.000000,0.627887,0.345602
3,2022090800,122,20,29,9,12.09,11.855248,0.980583,0.573149,203,...,38577,Bobby Wagner,ILB,47857,Devin Singletary,RB,0.380032,1.000000,0.980583,0.786872
4,2022090800,167,14,18,4,7.00,6.968414,0.995488,1.591540,191,...,43298,Leonard Floyd,DE,42489,Stefon Diggs,WR,0.308930,0.510575,0.951500,0.590335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6637,2022110700,3658,40,55,15,13.06,12.127753,0.928618,1.341343,211,...,44871,Tanoh Kpassagnon,DE,43362,Kenyan Drake,RB,0.670777,0.823322,0.928618,0.807572
6638,2022110700,3686,42,72,30,8.39,6.245190,0.744361,1.015135,211,...,48027,Kaden Elliss,OLB,43362,Kenyan Drake,RB,0.545607,1.000000,0.744361,0.763323
6639,2022110700,3707,39,54,15,9.29,9.080666,0.977467,1.452790,211,...,48027,Kaden Elliss,OLB,43362,Kenyan Drake,RB,0.511397,0.684012,0.977467,0.724292
6640,2022110700,3740,31,38,7,10.29,9.281939,0.902035,0.733757,211,...,48537,Carl Granderson,DE,43362,Kenyan Drake,RB,0.488677,1.000000,0.902035,0.796904


In [5]:
metric_df.columns

Index(['gameId', 'playId', 'contactFrameId', 'tackleFrameId', 'frames',
       'd_actual', 'd_ideal', 'd_eff', 'gap_tackle', 'w_carrier', 'w_tackler',
       's_downfield_delta', 's_contact', 's_downfield_contact',
       's_downfield_contact_t', 'tackle_category', 'nflId_t', 'displayName_t',
       'position_t', 'ballCarrierId', 'displayName', 'position',
       'drive_metric', 'tackle_gap_metric', 'vision_metric', 'overall_metric'],
      dtype='object')

In [6]:
metric_df.query('tackle_category=="proactive"').sort_values('overall_metric', ascending=False).iloc[0]

gameId                        2022100901
playId                               735
contactFrameId                        38
tackleFrameId                         51
frames                                13
d_actual                            8.95
d_ideal                         8.577797
d_eff                           0.958413
gap_tackle                      0.434166
w_carrier                            237
w_tackler                            295
s_downfield_delta              -5.372845
s_contact                           3.92
s_downfield_contact             3.083096
s_downfield_contact_t          -1.660834
tackle_category                proactive
nflId_t                            37104
displayName_t            Cameron Heyward
position_t                            DT
ballCarrierId                      46076
displayName                   Josh Allen
position                              QB
drive_metric                    0.959325
tackle_gap_metric                    1.0
vision_metric   

In [7]:
metric_df.query('tackle_category=="proactive"').sort_values('overall_metric', ascending=True).iloc[0]

gameId                        2022102000
playId                              2657
contactFrameId                        58
tackleFrameId                         69
frames                                11
d_actual                           11.09
d_ideal                         6.108158
d_eff                           0.550781
gap_tackle                       4.56838
w_carrier                            215
w_tackler                            195
s_downfield_delta               3.983389
s_contact                           8.33
s_downfield_contact             2.975702
s_downfield_contact_t          -1.447814
tackle_category                proactive
nflId_t                            54514
displayName_t             Alontae Taylor
position_t                            CB
ballCarrierId                      54666
displayName              Keaontay Ingram
position                              RB
drive_metric                    0.207786
tackle_gap_metric                    0.0
vision_metric   